##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 初心者のための TensorFlow 2.0 入門

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Note: これらのドキュメントは私たちTensorFlowコミュニティが翻訳したものです。コミュニティによる 翻訳は**ベストエフォート**であるため、この翻訳が正確であることや[英語の公式ドキュメント](https://www.tensorflow.org/?hl=en)の 最新の状態を反映したものであることを保証することはできません。 この翻訳の品質を向上させるためのご意見をお持ちの方は、GitHubリポジトリ[tensorflow/docs-l10n](https://github.com/tensorflow/docs-l10n)にプルリクエストをお送りください。 コミュニティによる翻訳やレビューに参加していただける方は、 [docs-ja@tensorflow.org メーリングリスト](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ja)にご連絡ください。

この短いイントロダクションでは [Keras](https://www.tensorflow.org/guide/keras/overview) を使って下記のことを行います。

1. 画像を分類するニューラルネットワークを構築する
2. このニューラルネットワークを訓練する
3. そして最後に、モデルの正解率を評価する

このファイルは [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) の notebook ファイルです。 Python プログラムはブラウザ上で直接実行されます。TensorFlow を学んだり使ったりするには最良の方法です。Google Colab のnotebook の実行方法は以下のとおりです。

1. Pythonランタイムへの接続：メニューバーの右上で「接続」を選択します。
2. ノートブックのコードセルをすべて実行：「ランタイム」メニューから「すべてのセルを実行」を選択します。

TensorFlow 2 のパッケージをダウンロードしてインストールします。プログラムに TensorFlow をインポートします。

Note: TensorFlow 2 パッケージをインストールするためには `pip` をアップグレードしてください。詳細は [インストール](https://www.tensorflow.org/install)を参照ください。

In [1]:
import tensorflow as tf

[MNIST データセット](http://yann.lecun.com/exdb/mnist/)をロードして準備します。サンプルを整数から浮動小数点数に変換します。

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 1s 0us/step


層を積み重ねて`tf.keras.Sequential`モデルを構築します。訓練のためにオプティマイザと損失関数を選びます。

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

モデルはそれぞれの標本について、クラスごとに"[ロジット](https://developers.google.com/machine-learning/glossary#logits)"や"[対数オッズ比](https://developers.google.com/machine-learning/glossary#log-odds)"と呼ばれるスコアを算出します。

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.06977536,  0.11707623,  0.15679863,  0.7047379 , -0.31973547,
        -0.5149855 ,  0.13406098, -0.13977441,  0.550864  ,  0.1178479 ]],
      dtype=float32)

`tf.nn.softmax` 関数はクラスごとにこれらのロジットを "確率" に変換します。

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.09254661, 0.09702934, 0.10096114, 0.17463097, 0.06268997,
        0.05157059, 0.09869143, 0.07505061, 0.14972512, 0.09710424]],
      dtype=float32)

Note: `tf.nn.softmax` をネットワークの最終レイヤーに対する活性化関数として組み込むことは可能です。これはモデルの出力に対して直接的に解釈できるようにする一方で、ソフトマックスの出力を用いる場合、すべてのモデルに対して正確で数値的安定に損失を計算することが不可能なため、このアプローチは推奨されません。

`losses.SparseCategoricalCrossentropy` 損失関数は、ロジットと `True` の インデックスに関するベクトルを入力にとり、それぞれの標本についてクラスごとに損失のスカラーを返します。

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

この損失は、クラスが正しい確率の対数をとって符号を反転させたものです。この値はモデルがこのクラスが正しいと確信しているときに 0 になります。

この訓練されていないモデルはランダムに近い確率 (それぞれのクラスについて 1/10) を出力します、最初の損失は `-tf.log(1/10) ~= 2.3` に近い値になるはずです。

In [7]:
loss_fn(y_train[:1], predictions).numpy()

2.9648037

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

`Model.fit` メソッドは損失を最小化するようにモデルのパラメータを調整します。

In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2960 - accuracy: 0.9141
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1409 - accuracy: 0.9583
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1029 - accuracy: 0.9689
Epoch 4/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0855 - accuracy: 0.9736
Epoch 5/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0725 - accuracy: 0.9772


`Model.evaluate` メソッドはモデルの性能を検査します。これには通常 "[検証用データセット](https://developers.google.com/machine-learning/glossary#validation-set)" または "[テストデータセット](https://developers.google.com/machine-learning/glossary#test-set)" を用います。

In [10]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0804 - accuracy: 0.9763


[0.08037655055522919, 0.9763000011444092]

この画像識別器はこのデータセットにおいておよそ 98% の正解率にまで訓練されました。更に学ぶためには [TensorFlow tutorials](https://www.tensorflow.org/tutorials/) を参照ください。

モデルが確率を返すようにしたい場合には、モデルをラップしてソフトマックス関数を適用してください。

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[3.06100759e-07, 3.87500032e-09, 1.65224610e-05, 3.68411274e-05,
        6.10613686e-12, 6.89580162e-08, 1.82639497e-14, 9.99943733e-01,
        8.92872890e-07, 1.66794393e-06],
       [1.43096486e-08, 1.14549715e-04, 9.99882102e-01, 2.01276384e-06,
        2.45202777e-16, 1.32523403e-09, 8.09930185e-08, 3.82063359e-15,
        1.21991263e-06, 5.42849689e-11],
       [3.31148868e-08, 9.99160051e-01, 2.58285232e-04, 1.28954798e-05,
        1.67805538e-05, 1.58413548e-06, 4.77477806e-06, 3.43940133e-04,
        2.00190305e-04, 1.51439042e-06],
       [9.99824941e-01, 6.36259170e-11, 5.32763588e-05, 2.90177900e-06,
        2.55561588e-07, 1.58756429e-05, 1.67017947e-06, 9.64636638e-05,
        8.45909938e-08, 4.53409939e-06],
       [5.43368287e-06, 1.80902793e-11, 1.08838940e-05, 9.07291664e-09,
        9.93671417e-01, 5.41697830e-07, 9.97074380e-07, 1.26915285e-04,
        6.48725745e-07, 6.18321123e-03]], dtype=float32)>